## Homework 03: due 22:00, Sunday, December 5, 2021

### -	Please work on this Jupyter notebook directly.
### -	Name a folder using your student id and Assignment number (e.g., b88501045-HW3), put all the python scripts and notebook files into the folder and zip the folder (e.g., b88501045-HW3.zip).
### - Submit the homework directly through the NTU COOL course website.


#### 0. Import essential modules

In [1]:
import scipy.stats as stats
import numpy as np
import random
import pandas as pd
import math
import matplotlib.pyplot as plt

#### 1. (30%) Consider the following dataset, sampled from a distribution H. We would like to know the confidence intervals for some of their given statistical properties.

In [2]:
samples = [28.82, 22.00, 24.89, 31.20, 29.34, 15.11, 24.75, 19.24, 19.48,
       22.05, 20.72, 27.27, 23.81, 20.61, 22.22, 21.67, 27.47, 18.97,
       21.57, 15.73,  7.24, 23.27, 24.32, 16.29, 31.35, 12.73, 20.23,
       19.06, 27.66, 27.35, 20.77, 21.89, 15.56, 10.10, 18.26, 20.78,
       26.15, 26.01, 18.06, 18.49]

#### Please complete the following tasks.

##### 1.1 (10%) Assuming that the population standard deviation is well estimated by the sample standard deviation, find the 95% (two-sided) confidence level for the population mean. 

In [3]:
sample_mean = np.mean(samples)
sample_stdev = np.std(samples,ddof=1)
n=len(samples)
population_stdev = sample_stdev/math.sqrt(n)

z_25 = stats.norm.ppf(0.025)
z_975 = stats.norm.ppf(0.975)

q_25 = sample_mean + z_25*population_stdev
q_975 = sample_mean + z_975*population_stdev

print('The lower confidence limit is', format(q_25,'.4f'))
print('The upper confidence limit is', format(q_975, '.4f'))

The lower confidence limit is 19.8922
The upper confidence limit is 23.2323


##### 1.2 (10%) Assuming that the population standard deviation is unknown, estimate the 95% (two-sided) confidence level for the population mean.

In [4]:
sample_mean = np.mean(samples)
sample_stdev = np.std(samples,ddof=1)
n=len(samples)
dof = n-1

t_25_dof = stats.t.ppf(0.025, df=dof)
t_975_dof = stats.t.ppf(0.975, df=dof)

q_25 = sample_mean + t_25_dof*sample_stdev/math.sqrt(n)
q_975 = sample_mean + t_975_dof*sample_stdev/math.sqrt(n)

print('The lower confidence limit is', format(q_25,'.4f'))
print('The upper confidence limit is', format(q_975, '.4f'))

The lower confidence limit is 19.8388
The upper confidence limit is 23.2857


##### 1.3 (10%) Find the 95% (two-sided) confidence level for the population standard deviation.

In [5]:
sample_stdev = np.mean(samples)
sample_stdev = np.std(samples,ddof=1)
sample_var = sample_stdev*sample_stdev
n=len(samples)
dof = n-1

z_025 = stats.chi2.ppf(0.025, df=dof)
z_975 = stats.chi2.ppf(0.975, df=dof)

q_025 = sample_var*(n-1)/z_025
q_975 = sample_var*(n-1)/z_975

#upper & lower換位置
print('The upper confidence limit is', format(math.sqrt(q_025),'.4f'))
print('The lower confidence limit is', format(math.sqrt(q_975), '.4f'))

The upper confidence limit is 6.9196
The lower confidence limit is 4.4144


#### 2. (30%) From the attached data file _ETH-Kobo_Jan.csv_, you can find historical January rainfall for 1993-2020 in Kobo, Ethiopia. Suppose that rainfall data follows the Gamma distribution, Gamma(α,β), where α=0.86 and β=74.28. Please complete the following tasks.

In [6]:
fn_data = "/Users/lintingxuan/Desktop/HW3/ETH-Kobo_Jan.csv"
# read csv file, a dataframe will be created to store data
df_rain_1d = pd.read_csv(fn_data)
kobo= df_rain_1d['Kobo'].tolist()
sample_mean = np.mean(kobo)
sample_stdev = np.std(kobo, ddof = 1)
sample_alpha=0.86
sample_beta=74.28

In [7]:
nBootstraps = 100
nSamples = len(kobo)

# random seed is set to 0 (or any other integer) to ensure the sampling result remain the same
np.random.seed(0)
# Convert sample list to a numpy array
np_samples = np.array(kobo)

# arrays to record sample means and standard deviations resulting from resampling
resamples_mean = np.zeros(nBootstraps)
resamples_std = np.zeros(nBootstraps)
resamples_alpha = np.zeros(nBootstraps)
resamples_beta = np.zeros(nBootstraps)


for b in range(nBootstraps):
    # numpy's random.choice is useful to randomly pick a given number of values 
    # (with replacement when replace is set to True) from the input array 
    resamples_b = np.random.choice(np_samples, size = nSamples, replace=True)
    
    # Compute mean and standard deviation from the resampled array
    mean_resamples_b = np.mean(resamples_b)
    std_resamples_b = np.std(resamples_b, ddof = 1)
    alpha_resamples_b,loc_resamples_b,beta_resamples_b=stats.gamma.fit(resamples_b, floc = 0.0)
    
    resamples_mean[b] = mean_resamples_b
    resamples_std[b] = std_resamples_b
    resamples_alpha[b] = alpha_resamples_b
    resamples_beta[b] = beta_resamples_b


##### 2.1 (15%) Find the 95% (two-sided) confidence level for the population mean using empirical bootstrapping (please set the number of bootstraps as 100).

In [8]:
# For empirical bootstrapping, we have to compute 'delta' 
# delta - the difference between the sample and resample targetted properties 

delta_mean =  resamples_mean - sample_mean
delta_std = resamples_std - sample_stdev


# Obtain the lower and upper CI limits
CI_mean_lower, CI_mean_upper = sample_mean - np.percentile(delta_mean,[97.5,2.5])
CI_std_lower, CI_std_upper = sample_stdev - np.percentile(delta_std,[97.5,2.5])


print('Two-sided 95% CIs for mean using empirical bootstrapping')
print('The lower limit of CI for mean =', format(CI_mean_lower, '.2f'))
print('The upper limit of CI for mean =', format(CI_mean_upper, '.2f'))

Two-sided 95% CIs for mean using empirical bootstrapping
The lower limit of CI for mean = 43.65
The upper limit of CI for mean = 83.68


##### 2.2	(15%) Find the 90% (two-sided) confidence level for the fitted parameters of the Gamma distribution using percentile bootstrapping (please set the number of bootstraps as 100).

In [9]:
# For the percentile bootstrapping, we just have to use resample taregetted properties.
delta_alpha = resamples_alpha - sample_alpha
delta_beta = resamples_beta - sample_beta

# Obtain the lower and upper CI limits
CI_alpha_lower, CI_alpha_upper = sample_alpha - np.percentile(delta_alpha,[95,5])
CI_beta_lower, CI_beta_upper = sample_beta - np.percentile(delta_beta,[95,5])


print('Two-sided 90% CIs for alpha using percentile bootstrapping')
print('The lower limit of CI for alpha =', format(CI_alpha_lower, '.2f'))
print('The upper limit of CI for alpha =', format(CI_alpha_upper, '.2f'))

print('Two-sided 90% CIs for beta using percentile bootstrapping')
print('The lower limit of CI for beta =', format(CI_beta_lower, '.2f'))
print('The upper limit of CI for beta =', format(CI_beta_upper, '.2f'))

Two-sided 90% CIs for alpha using percentile bootstrapping
The lower limit of CI for alpha = 0.12
The upper limit of CI for alpha = 1.08
Two-sided 90% CIs for beta using percentile bootstrapping
The lower limit of CI for beta = 52.50
The upper limit of CI for beta = 100.01


#### 3. (50%) Taiwan had done a great job in preventing local transmission of COVID-19 for over a year. However, due to a series of unfortunate events, local infections outbroke in May 2020. In the attached file _Covid19_TPE.csv_, you can find the records for the number of infected from 2021/05/15 to 2021/06/13 for Taipei City. Your task is to evaluate if the ‘soft’ lockdown introduced on 2021/05/15 by the Taipei City  work. More specifically, considering the 14-day incubation period of COVID-19, we want to compare if there is a decrease in the average number of infections before and after the 14-day incubation period (i.e., the potential effective date starting on 2021/05/29 in Taipei). Please perform the statstical tests under the following different scenarios.

In [10]:
fn_data = "/Users/lintingxuan/Desktop/HW3/Covid19_TPE.csv"
# read csv file, a dataframe will be created to store data
df_rain_1d = pd.read_csv(fn_data)
infect= df_rain_1d['Taipei_Infected'].tolist()
infect_1=infect[:14]
infect_2=infect[14:]

##### 3.1 (10%) Scenario 1 (asymmetric case): $\mu_A$ is known; $\mu_B$ is not known; $\sigma_B=\sigma_A = \sigma$

1. Null Hypothesis $ H_0: \mu_A = \mu_B$
2. Alternative Hypothesis $ H_1: \mu_B> \mu_A$
3. Test statistic $Z = \frac{\bar{X}_{B}-\mu_B}{\sigma/\sqrt{n}}$
4. Distribution of test statistic $Z \sim N(0,1)$
5. Critical Region $\Re$: $Z>z_\alpha$ (e.g. $\alpha=1$\%)
6. Evaluate $Z$ under $H_0$  & if $z_0>z_\alpha \Rightarrow$ Reject $H_0$
7. $p$-value: $p(Z>z_0)$

In [11]:
n_samples1 = len(infect_1)
n_samples2 = len(infect_2)
f_A_mean_sample = np.mean(infect_1)
mu_A = f_A_mean_sample
f_A_stdev_sample = np.std(infect_1, ddof=1)
sigma = f_A_stdev_sample

f_B_mean_sample = np.mean(infect_2)
mu_B = mu_A

#要加絕對值
Z = np.abs((f_B_mean_sample - mu_B)/(sigma/np.sqrt(n_samples1)))

alpha = 0.01
z_alpha = stats.norm.ppf(1-alpha)

print(f'test statistic Z = {Z:.4f}')
print(f'z_alpha = {z_alpha:.4f}')

if Z > z_alpha:
    p_value = 1-stats.norm.cdf(Z)
    print(f'We have to reject H0, and the p-value is {p_value:.4f}')
else:
    print('We have no ground to reject H0')

test statistic Z = 3.2934
z_alpha = 2.3263
We have to reject H0, and the p-value is 0.0005


##### 3.2 (10%) Scenario 2 (asymmetric case): $\mu_A$ is known; $\mu_B$ is not known; $\sigma_B$ is not known

1. Null Hypothesis $ H_0: \mu_A = \mu_B$
2. Alternative Hypothesis $ H_1: \mu_B> \mu_A$
3. Test statistic $T = \frac{\bar{X}_{B}-\mu_B}{\hat{S_x}_B/\sqrt{n}}$
4. Distribution of test statistic $T \sim t(n-1)$
5. Critical Region $\Re$: $T>t_{n-1,\alpha}$ (e.g. $\alpha=1$\%)
6. Evaluate $T$ under $H_0$  & if $t_0>t_{n-1,\alpha} \Rightarrow$ Reject $H_0$
7. $p$-value: $p(T>t_0)$

In [16]:
f_A_mean_sample = np.mean(infect_1)
mu_A = f_A_mean_sample

f_B_mean_sample = np.mean(infect_2)
mu_B = mu_A
f_B_stdev_sample = np.std(infect_2, ddof=1)

#要加絕對值
T = np.abs((f_B_mean_sample - mu_B)/(f_B_stdev_sample/np.sqrt(n_samples2)))

df = len(infect_2)-1

alpha = 0.01
t_alpha = stats.t.ppf(1-alpha,df)

print(f'test statistic T = {T:.4f}')
print(f't_alpha = {t_alpha:.4f}')

if T > t_alpha:
    p_value = 1-stats.t.cdf(T,df)
    print(f'We have to reject H0, and the p-value is {p_value:.4f}')
else:
    print('We have no ground to reject H0')

test statistic T = 5.2536
t_alpha = 2.6025
We have to reject H0, and the p-value is 0.0000


##### 3.3 (10%) Scenario 3 (symmetric case): $\mu_A$ is not known; $\mu_B$ is not known; $\sigma_B$ is known; $\sigma_A$ is known
1. Null Hypothesis: $ H_0: \mu_A = \mu_B$
2. Alternative Hypothesis: $ H_1: \mu_A \neq \mu_B$
3. Test statistic: $Z = \frac{(\bar{X}_A - \bar{X}_B)-(\mu_A-\mu_B)}{\sigma_{{\bar{X}_A} -{\bar{X}_B}} }$
4. Distribution of test statistic: $Z \sim N(0,1)$
5. Critical Region $\Re$: $|Z|>z_{\alpha/2}$ (e.g. $\alpha=5$\%)
6. Evaluate $Z$ under $H_0$: if $|z_0|>z_{\alpha/2} \Rightarrow$ Reject $H_0$
7. $p$-value & $p(|Z|>z_0)$

where $\sigma_{{\bar{X}_A} -{\bar{X}_B}} = \sqrt{\frac{\sigma_A^2}{n_A} +\frac {\sigma_B^2}{n_B} }$ 

In [17]:
f_A_mean_sample = np.mean(infect_1)
f_A_stdev_sample = np.std(infect_1, ddof=1)
sigma_A = f_A_stdev_sample

f_B_mean_sample = np.mean(infect_2)
f_B_stdev_sample = np.std(infect_2, ddof=1)
sigma_B = f_B_stdev_sample

sigma_A_B = np.sqrt((sigma_A*sigma_A/n_samples1) + (sigma_B*sigma_B/n_samples2))

Z = np.abs((f_B_mean_sample - f_A_mean_sample)/sigma_A_B)

alpha = 0.01
z_alpha = stats.norm.ppf(1-alpha)

print(f'test statistic Z = {Z:.4f}')
print(f'z_alpha = {z_alpha:.4f}')

if Z > z_alpha:
    p_value = 1-stats.norm.cdf(Z)
    print(f'We have to reject H0, and the p-value is {p_value:.4f}')
else:
    print('We have no ground to reject H0')

test statistic Z = 2.7905
z_alpha = 2.3263
We have to reject H0, and the p-value is 0.0026


##### 3.4 (10%) Scenario 4 (symmetric case): $\mu_A$ is not known; $\mu_B$ is not known; $\sigma_B= \sigma_A = \sigma$ is not known
1. Null Hypothesis: $ H_0: \mu_A = \mu_B$
2. Alternative Hypothesis: $ H_1: \mu_A > \mu_B$
3. Test statistic: $t = \frac{ (\bar{X}_A - \bar{X}_B)-(\mu_A-\mu_B)} {\widehat{S}_{X_A,X_B} \sqrt{\frac{1}{n_A} +\frac{1}{n_B} } } $
4. Distribution of test statistic: $T \sim t(n_A +n_B -2)$
5. Critical Region $\Re$: $T>t_{n_A+n_B-2,\alpha}$
6. Evaluate $T$ under $H_0$:if $t_0>t_{n_A+n_B-2,\alpha} \Rightarrow$ Reject $H_0$
7. $p$-value: $p(T>t_0)$

where $\widehat{S}_{X_A,X_B}^2 =  \frac{(n_A-1) \widehat{S}_{X_A}^2 +(n_B-1) \widehat{S}_{X_B}^2  }{n_A + n_B - 2}$

In [14]:
f_A_mean_sample = np.mean(infect_1)
f_A_stdev_sample = np.std(infect_1, ddof=1)

f_B_mean_sample = np.mean(infect_2)
f_B_stdev_sample = np.std(infect_2, ddof=1)

stdev_sample_A_B = np.sqrt((((n_samples1-1)*f_A_stdev_sample*f_A_stdev_sample) + ((n_samples2-1)*f_B_stdev_sample*f_B_stdev_sample))/(n_samples1+n_samples2-2))

T = np.abs((f_A_mean_sample - f_B_mean_sample))/(stdev_sample_A_B*np.sqrt(1/n_samples1+1/n_samples2))

df = n_samples2+n_samples1-2

alpha = 0.01
t_alpha = stats.t.ppf(1-alpha,df)

print(f'test statistic T = {T:.4f}')
print(f't_alpha = {t_alpha:.4f}')

p_value = 1-stats.t.cdf(T,df)

if T > t_alpha:
    
    print(f'We have to reject H0, and the p-value is {p_value:.4f}')
else:
    print(f'We have no ground to reject H0, and the p-value is {p_value:.4f}')

test statistic T = 2.8648
t_alpha = 2.4671
We have to reject H0, and the p-value is 0.0039


##### 3.5 (10%) Scenario 5 (symmetric case): $\mu_A$ is not known; $\mu_B$ is not known; $\sigma_B$ is not known; $\sigma_A$ is not known
1. Null Hypothesis: $ H_0: \mu_A = \mu_B$
2. Alternative Hypothesis: $ H_1: \mu_A > \mu_B$
3. Test statistic: $R = \frac{ (\bar{X}_A - \bar{X}_B)-(\mu_A-\mu_B)} { \sqrt{\frac{\hat{S_x}_{A}^2 }{n_A} +\frac{\hat{S_x}_{B}^2}{n_B} } }  $
4. Distribution of test statistic: $R \sim t(\nu)$
5. Critical Region $\Re$: $R>t_{\nu,\alpha}$
6. Evaluate $R$ under $H_0$: if $r_0>t_{\nu,\alpha} \Rightarrow$ Reject $H_0$
7. $p$-value: $p(R>r_0)$

$\nu \approx  \left( \frac{\hat{S_x}_{A}^2 }{n_A} +\frac{\hat{S_x}_{B}^2}{n_B} \right)^2 / \left[ \frac{1}{n_A-1} \left( \frac{\hat{S_x}_{A}^2 }{n_A} \right)^2 + \frac{1}{n_B-1} \left( \frac{\hat{S_x}_{B}^2 }{n_B} \right)^2 \right]$


In [15]:
f_A_mean_sample = np.mean(infect_1)
f_A_stdev_sample = np.std(infect_1, ddof=1)

f_B_mean_sample = np.mean(infect_2)
f_B_stdev_sample = np.std(infect_2, ddof=1)

S_xA_n = f_A_stdev_sample*f_A_stdev_sample/len(infect_1)
S_xB_n = f_B_stdev_sample*f_B_stdev_sample/len(infect_2)

R = np.abs((f_A_mean_sample - f_B_mean_sample))/np.sqrt(S_xA_n + S_xB_n)

nu = (S_xA_n+S_xB_n)**2/(S_xA_n*S_xA_n/(n_samples1-1) + S_xB_n*S_xB_n/(n_samples2-1))

alpha = 0.01
t_alpha = stats.t.ppf(1-alpha,nu)

print(f'test statistic T = {R:.4f}')
print(f't_alpha = {t_alpha:.4f}')

p_value = 1-stats.t.cdf(R,nu)

if R > t_alpha:
    
    print(f'We have to reject H0, and the p-value is {p_value:.4f}')
else:
    print(f'We have no ground to reject H0, and the p-value is {p_value:.4f}')

test statistic T = 2.7905
t_alpha = 2.5062
We have to reject H0, and the p-value is 0.0053
